# Transformer

Transformer의 핵심 구조인 Multi-head Attention을 구현하는 실습입니다.
1. Multi-head attention 및 self-attention 구현.
2. 각 과정에서 일어나는 연산과 input/output 형태 이해.

### 필요 패키지 import

In [1]:
from torch import nn
from torch.nn import functional as F
from tqdm import tqdm

import torch
import math

## Req. 2-1 Multi-head self-attention 구조 익히기

### **데이터 전처리**
vocab_size 100인 가상의 시퀀스 데이터를 생성합니다.

각 데이터에 할당된 숫자는 tokenizing과 정수화가 이뤄진 형태입니다.

In [2]:
pad_id = 0
vocab_size = 100

data = [
  [62, 13, 47, 39, 78, 33, 56, 13, 39, 29, 44, 86, 71, 36, 18, 75],
  [60, 96, 51, 32, 90],
  [35, 45, 48, 65, 91, 99, 92, 10, 3, 21, 54],
  [75, 51],
  [66, 88, 98, 47],
  [21, 39, 10, 64, 21],
  [98],
  [77, 65, 51, 77, 19, 15, 35, 19, 23, 97, 50, 46, 53, 42, 45, 91, 66, 3, 43, 10],
  [70, 64, 98, 25, 99, 53, 4, 13, 69, 62, 66, 76, 15, 75, 45, 34],
  [20, 64, 81, 35, 76, 85, 1, 62, 8, 45, 99, 77, 19, 43]
]

In [3]:
# 길이 맞춰주기 위해 패딩합니다.
def padding(data):
  max_len = len(max(data, key=len))
  print(f"Maximum sequence length: {max_len}")

  for i, seq in enumerate(tqdm(data)):
    if len(seq) < max_len:
      data[i] = seq + [pad_id] * (max_len - len(seq))

  return data, max_len

In [4]:
data, max_len = padding(data)

Maximum sequence length: 20


100%|██████████| 10/10 [00:00<00:00, 24995.85it/s]


In [7]:
for d in data:

    print(d)

[62, 13, 47, 39, 78, 33, 56, 13, 39, 29, 44, 86, 71, 36, 18, 75, 0, 0, 0, 0]
[60, 96, 51, 32, 90, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[35, 45, 48, 65, 91, 99, 92, 10, 3, 21, 54, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[75, 51, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[66, 88, 98, 47, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[21, 39, 10, 64, 21, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[98, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[77, 65, 51, 77, 19, 15, 35, 19, 23, 97, 50, 46, 53, 42, 45, 91, 66, 3, 43, 10]
[70, 64, 98, 25, 99, 53, 4, 13, 69, 62, 66, 76, 15, 75, 45, 34, 0, 0, 0, 0]
[20, 64, 81, 35, 76, 85, 1, 62, 8, 45, 99, 77, 19, 43, 0, 0, 0, 0, 0, 0]


### Hyperparameter 세팅 및 embedding

In [8]:
d_model = 512  # model의 hidden size
num_heads = 8  # head의 개수

# d_model이 입력을 projection 시킬 임베딩 space의 차원이므로, num_heads로 나누어 떨어져야 한다.

In [9]:
embedding = nn.Embedding(vocab_size, d_model)

# B: batch size, L: maximum sequence length
batch = torch.LongTensor(data)  # (B, L)
batch_emb = embedding(batch)  # (B, L, d_model)

In [10]:
print(batch_emb)
print(batch_emb.shape)

tensor([[[ 1.0694, -0.2390,  0.1662,  ..., -1.9457,  0.0275,  2.1537],
         [-0.5915, -0.0119, -0.1197,  ..., -0.2442,  0.0591,  0.6294],
         [ 1.6521,  0.2287,  0.0495,  ..., -0.1128, -0.1242,  0.3815],
         ...,
         [ 0.7478, -1.0914, -1.9780,  ...,  0.1879, -0.8365, -0.0996],
         [ 0.7478, -1.0914, -1.9780,  ...,  0.1879, -0.8365, -0.0996],
         [ 0.7478, -1.0914, -1.9780,  ...,  0.1879, -0.8365, -0.0996]],

        [[-1.0447, -0.0719,  0.1134,  ...,  0.8984,  0.1344, -0.0302],
         [ 0.7037,  2.7984, -0.7042,  ..., -0.1699,  0.1612, -0.4142],
         [-1.2631,  0.6691,  0.0374,  ..., -1.6806,  0.2675,  0.1831],
         ...,
         [ 0.7478, -1.0914, -1.9780,  ...,  0.1879, -0.8365, -0.0996],
         [ 0.7478, -1.0914, -1.9780,  ...,  0.1879, -0.8365, -0.0996],
         [ 0.7478, -1.0914, -1.9780,  ...,  0.1879, -0.8365, -0.0996]],

        [[ 2.5548, -1.0154,  0.7586,  ...,  0.0159,  1.5879,  0.4343],
         [ 0.0770,  0.6727,  1.5715,  ..., -0

### Linear projection & 여러 head로 나누기

Multi-head attention 내에서 쓰이는 linear projection matrix들을 정의합니다.

In [11]:
w_q = nn.Linear(d_model, d_model)
w_k = nn.Linear(d_model, d_model)
w_v = nn.Linear(d_model, d_model)

In [12]:
w_0 = nn.Linear(d_model, d_model)

In [13]:
q = w_q(batch_emb)  # (B, L, d_model)
k = w_k(batch_emb)  # (B, L, d_model)
v = w_v(batch_emb)  # (B, L, d_model)

print(q.shape)
print(k.shape)
print(v.shape)

torch.Size([10, 20, 512])
torch.Size([10, 20, 512])
torch.Size([10, 20, 512])


Q, k, v를 `num_head`개의 차원 분할된 여러 vector로 만듭니다.

- 이론적으로는 multi-head attention을 수행하면 input을 각각 다른 head 개수만큼의 Wq, Wk, Wv로 linear transformation 해서 각각 여러번의 attention 수행한 후 concat 한 후 linear transformation 수행해준다
- 구현에서는 Wq, Wk, Wv 한 개씩
- 실제 `attention is all you need` 논문의 구현 예시는 Query vector 한개를 dim으로 쪼개서 진행한다

In [14]:
batch_size = q.shape[0]
d_k = d_model // num_heads

# num_heads * d_k로 쪼갠다
q = q.view(batch_size, -1, num_heads, d_k)  # (B, L, num_heads, d_k)
k = k.view(batch_size, -1, num_heads, d_k)  # (B, L, num_heads, d_k)
v = v.view(batch_size, -1, num_heads, d_k)  # (B, L, num_heads, d_k)

print(q.shape)
print(k.shape)
print(v.shape)

torch.Size([10, 20, 8, 64])
torch.Size([10, 20, 8, 64])
torch.Size([10, 20, 8, 64])


In [15]:
# num_heads를 밖으로 뺌으로써
# 각 head가 (L, d_k) 만큼의 matrix를 가지고 self-attention 수행

q = q.transpose(1, 2)  # (B, num_heads, L, d_k)
k = k.transpose(1, 2)  # (B, num_heads, L, d_k)
v = v.transpose(1, 2)  # (B, num_heads, L, d_k)

print(q.shape)
print(k.shape)
print(v.shape)

torch.Size([10, 8, 20, 64])
torch.Size([10, 8, 20, 64])
torch.Size([10, 8, 20, 64])


### Scaled dot-product self-attention 구현

각 head에서 실행되는 self-attetion 과정입니다.

In [16]:
# shape - (L, L)
# 같은 sequence 내에 서로 다른 token들에게 얼마나 가중치를 두고 attention을 해야하는가
attn_scores = torch.matmul(q, k.transpose(-2, -1)) / math.sqrt(d_k)  # (B, num_heads, L, L)
# softmax - row-wise이기 때문에 dim은 -1
attn_dists = F.softmax(attn_scores, dim=-1)  # (B, num_heads, L, L)

print(attn_dists)
print(attn_dists.shape)

tensor([[[[0.0321, 0.0377, 0.0567,  ..., 0.0488, 0.0488, 0.0488],
          [0.0532, 0.0586, 0.0367,  ..., 0.0577, 0.0577, 0.0577],
          [0.0503, 0.0501, 0.0560,  ..., 0.0578, 0.0578, 0.0578],
          ...,
          [0.0939, 0.0711, 0.0937,  ..., 0.0272, 0.0272, 0.0272],
          [0.0939, 0.0711, 0.0937,  ..., 0.0272, 0.0272, 0.0272],
          [0.0939, 0.0711, 0.0937,  ..., 0.0272, 0.0272, 0.0272]],

         [[0.0914, 0.0527, 0.0437,  ..., 0.0480, 0.0480, 0.0480],
          [0.0405, 0.0613, 0.0348,  ..., 0.0277, 0.0277, 0.0277],
          [0.0641, 0.0758, 0.0420,  ..., 0.0511, 0.0511, 0.0511],
          ...,
          [0.0516, 0.0497, 0.0692,  ..., 0.0541, 0.0541, 0.0541],
          [0.0516, 0.0497, 0.0692,  ..., 0.0541, 0.0541, 0.0541],
          [0.0516, 0.0497, 0.0692,  ..., 0.0541, 0.0541, 0.0541]],

         [[0.0291, 0.0744, 0.0439,  ..., 0.0448, 0.0448, 0.0448],
          [0.0710, 0.0954, 0.0466,  ..., 0.0309, 0.0309, 0.0309],
          [0.0601, 0.0342, 0.0635,  ..., 0

In [17]:
attn_values = torch.matmul(attn_dists, v)  # (B, num_heads, L, d_k)

print(attn_values.shape)

torch.Size([10, 8, 20, 64])


### 각 head의 결과물 병합

각 head의 결과물을 concat하고 동일 차원으로 linear projection합니다.

In [18]:
attn_values = attn_values.transpose(1, 2)  # (B, L, num_heads, d_k)
attn_values = attn_values.contiguous().view(batch_size, -1, d_model)  # (B, L, d_model)

print(attn_values.shape)

torch.Size([10, 20, 512])


In [19]:
# w_0 : (d_model, d_model)
# 서로 다른 의미로 foucsing 된 각 head의 self-attention 정보들을 합쳐주는 역할 수행
outputs = w_0(attn_values)

print(outputs)
print(outputs.shape)

tensor([[[-0.1165,  0.1298,  0.0666,  ..., -0.1141,  0.0476,  0.0787],
         [-0.1516,  0.1034,  0.0816,  ..., -0.1275,  0.0539,  0.1626],
         [-0.1476,  0.0405, -0.0229,  ..., -0.0741,  0.0741,  0.0630],
         ...,
         [-0.1154,  0.0567,  0.0332,  ..., -0.0294,  0.0969,  0.0368],
         [-0.1154,  0.0567,  0.0332,  ..., -0.0294,  0.0969,  0.0368],
         [-0.1154,  0.0567,  0.0332,  ..., -0.0294,  0.0969,  0.0368]],

        [[-0.5060,  0.2339,  0.2490,  ..., -0.4742,  0.0375,  0.5349],
         [-0.5093,  0.2369,  0.1996,  ..., -0.4750,  0.0638,  0.5631],
         [-0.5939,  0.2865,  0.2446,  ..., -0.4908,  0.0479,  0.5590],
         ...,
         [-0.4932,  0.2491,  0.2009,  ..., -0.4722,  0.0540,  0.5183],
         [-0.4932,  0.2491,  0.2009,  ..., -0.4722,  0.0540,  0.5183],
         [-0.4932,  0.2491,  0.2009,  ..., -0.4722,  0.0540,  0.5183]],

        [[-0.3577,  0.2886,  0.0669,  ..., -0.3673,  0.0200,  0.2743],
         [-0.3638,  0.3230,  0.1383,  ..., -0

## Req. 2-2 Multi-head self-attention 모듈 클래스 구현

위의 과정을 모두 합쳐 하나의 Multi-head attention 모듈 class를 구현하겠습니다.

아래 코드의 TODO 부분을 채워주세요.

In [20]:
class MultiheadAttention(nn.Module):
  def __init__(self):
    super(MultiheadAttention, self).__init__()

    # Q, K, V learnable matrices
    self.w_q = nn.Linear(d_model, d_model)
    self.w_k = nn.Linear(d_model, d_model)
    self.w_v = nn.Linear(d_model, d_model)

    # Linear projection for concatenated outputs
    self.w_0 = nn.Linear(d_model, d_model)

  # scaled-dot product attention
  def self_attention(self, q, k, v):
    attn_scores = torch.matmul(q, k.transpose(-2, -1)) / math.sqrt(d_k)  # (B, num_heads, L, L)
    attn_dists = F.softmax(attn_scores, dim=-1)  # (B, num_heads, L, L)

    attn_values = torch.matmul(attn_dists, v)  # (B, num_heads, L, d_k)

    return attn_values

  def forward(self, q, k, v):
    batch_size = q.shape[0]

    # linear projection
    ################################################################################
    # TODO 1: Implement the forward pass for linear projection.                #
    ################################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    q = w_q(q)  # (B, L, d_model)
    k = w_k(k)  # (B, L, d_model)
    v = w_v(v)  # (B, L, d_model)

    # head만큼 쪼개준다
    ################################################################################
    # TODO 2: Implement the forward pass for split head.                #
    ################################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    d_k = d_model // num_heads

    # num_heads * d_k로 쪼갠다
    q = q.view(batch_size, -1, num_heads, d_k)  # (B, L, num_heads, d_k)
    k = k.view(batch_size, -1, num_heads, d_k)  # (B, L, num_heads, d_k)
    v = v.view(batch_size, -1, num_heads, d_k)  # (B, L, num_heads, d_k)


    # 각 head가 (L, d_k)의 matrix를 담당하도록 만든다
    q = q.transpose(1, 2)  # (B, num_heads, L, d_k)
    k = k.transpose(1, 2)  # (B, num_heads, L, d_k)
    v = v.transpose(1, 2)  # (B, num_heads, L, d_k)

    attn_values = self.self_attention(q, k, v)  # (B, num_heads, L, d_k)
    attn_values = attn_values.transpose(1, 2).contiguous().view(batch_size, -1, d_model)  # (B, L, num_heads, d_k) => (B, L, d_model)

    return self.w_0(attn_values)

In [21]:
multihead_attn = MultiheadAttention()

outputs = multihead_attn(batch_emb, batch_emb, batch_emb)  # (B, L, d_model)

In [22]:
print(outputs)
print(outputs.shape)  # (batch_size, length, d_model)

tensor([[[ 0.0084,  0.0697,  0.0066,  ...,  0.0762, -0.1163, -0.1509],
         [ 0.0095,  0.0896,  0.0087,  ...,  0.0376, -0.0603, -0.2201],
         [-0.0065,  0.0732,  0.0640,  ...,  0.0287, -0.1010, -0.2239],
         ...,
         [-0.0384,  0.1396,  0.1162,  ...,  0.1014, -0.1325, -0.2189],
         [-0.0384,  0.1396,  0.1162,  ...,  0.1014, -0.1325, -0.2189],
         [-0.0384,  0.1396,  0.1162,  ...,  0.1014, -0.1325, -0.2189]],

        [[-0.2871,  0.2953, -0.3290,  ...,  0.0831,  0.1059, -0.2935],
         [-0.1951,  0.2030, -0.2173,  ...,  0.0829,  0.1585, -0.2751],
         [-0.2699,  0.3090, -0.2791,  ...,  0.1213,  0.1017, -0.2839],
         ...,
         [-0.2669,  0.2389, -0.2369,  ...,  0.0907,  0.0859, -0.2880],
         [-0.2669,  0.2389, -0.2369,  ...,  0.0907,  0.0859, -0.2880],
         [-0.2669,  0.2389, -0.2369,  ...,  0.0907,  0.0859, -0.2880]],

        [[-0.2609,  0.3197, -0.1531,  ...,  0.1359, -0.0250, -0.2770],
         [-0.1726,  0.2895, -0.1467,  ...,  0